In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
import re
import io
from google.colab import files
from collections import Counter

Using TensorFlow backend.


In [0]:
#dataset = pd.read_csv('imdb_master.csv', encoding="ISO-8859-1")
uploaded = files.upload()
dataset = pd.read_csv(io.StringIO(uploaded['imdb_master.csv'].decode('ISO-8859-1')))

Saving imdb_master.csv to imdb_master.csv


In [0]:
trainingSet = dataset[dataset['type']=='train'].values
testingSet = dataset[dataset['type']=='test'].values
inputTrain = trainingSet[:,2]
inputTest = testingSet[:,2]
outputTrain = trainingSet[:,3]
outputTest = testingSet[:,3]

In [0]:
def PreprocessIntConversion(review):
    
    review = re.sub('<[a-z]* />','',review)
    review = re.sub('[()-.:",\'/!?]','',review)
    words = review.split()
    counts = Counter(words)
    #print(counts)
    vocab = sorted(counts,key = counts.get,reverse=True)
    #print(vocab)
    vocabToInt = {word: w for w, word in enumerate(vocab,1)}
    #print(vocabToInt)
    review_int = []
    for w in words:
        review_int.append(vocabToInt[w])
    #print(review_int)
    return review_int

In [0]:
strToIntinputTrain = [PreprocessIntConversion(x) for x in inputTrain]
strToIntOutputTrain = [1 if x == 'pos' else 0 for x in outputTrain ]
strToIntInputTest = [PreprocessIntConversion(x) for x in inputTest]
strToIntOutputTest = [1 if x == 'pos' else 0 for x in outputTest ]

In [0]:
strToIntinputTrain = sequence.pad_sequences(strToIntinputTrain,maxlen = 200)
strToIntInputTest = sequence.pad_sequences(strToIntInputTest,maxlen = 200)
#vocabSize = calculateVocabSize(strToIntinputTrain,strToIntInputTest)
print(strToIntinputTrain[0])

[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 13  2  1 14 15 16 17 18
 19  1 20 21 22  3  1 23 24 25  5  1 26 27  2  6 28 29 30 31 32  5  7 33
 34 35 36 37  8  4 38 39  2  9 40 41 10 42  6  4 43 44  3 45 46 47 48 49
 10 50 51 11 52 53 54 55  4 56 57 12  7 11 58 59 60 61 62 63 64 65 66  1
 67 68 69  1 70 71  9 72 73 74 75 76  3 77 78 79  8 80 81 82 83 84 85 86
 87 88 89 90 91 12 92 93]


In [0]:
model = Sequential()
model.add(Embedding(1050,128))
model.add(LSTM(128,dropout = 0.2,recurrent_dropout = 0.2))
model.add(Dense(1,activation = 'sigmoid'))

In [0]:
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy'])

In [0]:
model.fit(strToIntinputTrain,strToIntOutputTrain,batch_size = 100, epochs = 20, verbose = 2, validation_data = (strToIntInputTest,strToIntOutputTest))
model.save('my_model.h5')

In [0]:
score, accuracy = model.evaluate(strToIntInputTest,strToIntOutputTest,batch_size = 100,verbose = 2)
print("Test score is {}".format(score))
print("Accuracy is {}".format(accuracy))

Test score is 1.0640472539663315
Accuracy is 0.5009600016921759


In [0]:
from keras.models import load_model
model = load_model('my_model.h5')

In [0]:
score, accuracy = model.evaluate(strToIntInputTest,strToIntOutputTest,batch_size = 100,verbose = 2)
print("Test score is {}".format(score))
print("Accuracy is {}".format(accuracy))

Test score is 1.0168919954299926
Accuracy is 0.5006000015884637
